In [1]:
### Find Missing Utilities 
# These are utilities that aren't in EIA923 2015 Generation & Fuel table, which
# was used to make the initial list of utilities

In [2]:
import sys, os
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
sys.path.append('/home/alana/Dropbox/catalyst/pudl/')
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia923, models_eia923
from pudl.eia923 import get_eia923_page
from pudl.eia923 import get_eia923_xlsx, get_eia923_column_map
from pudl.eia923 import get_eia923_plants
from pudl.pudl import *
from pudl.constants import tab_map_eia923
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [3]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.eia923
%aimport pudl.constants
%aimport pudl.settings
%aimport pudl.models
%aimport pudl.models_eia923

In [4]:
eia923_years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [5]:
eia923_xlsx = get_eia923_xlsx(eia923_years)

Reading EIA 923 spreadsheet data for 2009.
Reading EIA 923 spreadsheet data for 2010.
Reading EIA 923 spreadsheet data for 2011.
Reading EIA 923 spreadsheet data for 2012.
Reading EIA 923 spreadsheet data for 2013.
Reading EIA 923 spreadsheet data for 2014.
Reading EIA 923 spreadsheet data for 2015.
Reading EIA 923 spreadsheet data for 2016.


In [6]:
eia923_plants = get_eia923_plants(eia923_years, eia923_xlsx)
eia923_operator_ids = eia923_plants[['operator_id']]

Converting EIA 923 plant_frame to DataFrame...
Converting EIA 923 generation_fuel to DataFrame...
Converting EIA 923 boiler_fuel to DataFrame...
Converting EIA 923 generator to DataFrame...
Converting EIA 923 fuel_receipts_costs to DataFrame...


In [7]:
eia923_plants = eia923_plants.dropna(subset=['operator_id'])
eia923_plants.head(2)

,plant_id,plant_name,plant_state,combined_heat_power,eia_sector,naics_code,reporting_frequency,nameplate_capacity_mw,operator_name,operator_id,census_region,nerc_region
0,2,Bankhead Dam,AL,N,1.0,22.0,A,45.0,Alabama Power Co,195.0,ESC,SERC
1,3,Barry,AL,N,1.0,22.0,M,2841.5,Alabama Power Co,195.0,ESC,SERC


In [8]:
pwd

'/home/alana/Dropbox/catalyst/pudl/results/notebooks'

In [62]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'utilities_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc': int,
                                          'respondent_name_ferc': str,
                                          'plant_name_ferc': str,
                                          'plant_id_eia': int,
                                          'plant_name_eia': str,
                                          'operator_name_eia': str,
                                          'operator_id_eia': int})
output_operator_ids = output_tab_plants[['operator_id_eia', 'operator_name_eia']]
output_operator_ids = output_operator_ids.dropna()

In [63]:
output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia': 'operator_id'})

In [64]:
merged = eia923_operator_ids.merge(output_operator_ids, indicator=True, how='outer')
merged = merged[merged['_merge'] == 'right_only']
merged = merged.drop_duplicates()
del merged['_merge']

In [65]:
merged.shape

(513, 2)

In [94]:
missing_operators = merged.rename(columns={0: 'operator_id',})
missing_operators.drop_duplicates()
eia923_plants.drop_duplicates('operator_id')
missing_operators["operator_id"] = missing_operators.operator_id.astype(int)
eia923_plants["operator_id"] = eia923_plants.operator_id.astype(int)

In [95]:
missing_operators = pd.merge(missing_operators, eia923_plants, on=['operator_id'], how='left')

In [96]:
missing_operators.shape

(513, 13)

In [97]:
# output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
#                               converters={'plant_id': int,
#                                           'plant_name': str,
#                                           'respondent_id_ferc': int,
#                                           'respondent_name_ferc': str,
#                                           'plant_name_ferc': str,
#                                           'plant_id_eia': int,
#                                           'plant_name_eia': str,
#                                           'operator_name_eia': str,
#                                           'operator_id_eia': int})
# output_operator_ids = output_tab_plants[['operator_id_eia', 'operator_name_eia']]
# output_operator_ids = output_operator_ids.dropna()

In [98]:
# output_operator_ids = output_operator_ids.rename(columns={'operator_id_eia': 'operator_id'})

In [99]:
missing_operators = pd.merge(missing_operators, output_operator_ids, on=['operator_id'], how='left')

In [100]:
missing_operators = missing_operators.drop_duplicates('operator_id')
# del missing_operators['operator_id']


In [101]:
missing_operators.head()

,operator_id,operator_name_eia_x,plant_id,plant_name,plant_state,combined_heat_power,eia_sector,naics_code,reporting_frequency,nameplate_capacity_mw,operator_name,census_region,nerc_region,operator_name_eia_y
0,57150,Bethel Wind Energy LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bethel Wind Energy LLC
1,56886,Big Sandy Peaker Plant LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Big Sandy Peaker Plant LLC
2,2338,Calpine Central LP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calpine Central LP
4,56744,Elk Wind Energy LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elk Wind Energy LLC
5,6844,FPLE Forney LP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FPLE Forney LP


In [105]:
missing_operators = missing_operators[['operator_id', 'operator_name_eia_x']]
missing_operators = missing_operators.rename(columns={'operator_name_eia_x': 'operator_name'})

In [106]:
# missing_operators = missing_operators[list(missing_operators.columns[:2])]

In [107]:
missing_operators.head()

,operator_id,operator_name
0,57150,Bethel Wind Energy LLC
1,56886,Big Sandy Peaker Plant LLC
2,2338,Calpine Central LP
4,56744,Elk Wind Energy LLC
5,6844,FPLE Forney LP


In [ ]:
# missing_operators.to_csv('../id_mapping/missing_operators.csv', index=False)